In [12]:
import pandas as pd
import re
import json

## Tairement de la donnée

In [3]:
df = pd.read_json("data.json")
df.set_index("index",inplace=True)
df.shape


(5, 11)

In [4]:
df[df.duplicated("image")]["name"]


index
1    Pancook Lille Zénith
2             La Fossetta
4                 Quai 38
Name: name, dtype: object

In [5]:
df["type"]

index
0       Restaurant italien
1      Restaurant français
2       Restaurant italien
3               Restaurant
4    Restaurant de poisson
Name: type, dtype: object

In [6]:
df["googleMapRating"] = df["googleMapRating"].apply(lambda x : float(x.split("(")[0].replace(",",".")))
df["price"] = df["price"].apply(lambda x : len(x) if x is not None else 0)
df["coordonne"] = df["gmapLink"].apply(lambda x : re.findall(r"[0-9]{1,3}\.[0-9]*!4d[0-9]{1,3}\.[0-9]*",x)[0].split("!4d"))
df["lat"] = df["coordonne"].apply(lambda x: x[0])
df["long"] = df["coordonne"].apply(lambda x: x[1])
df["phone"] = df["info"].apply(lambda x : re.search(r"([0-9]{2} ){4}[0-9]{2}",x).group().replace(" ",""))



In [7]:
def getHorraires(x):
    result = {}
    for e in x :
        splited = e.split(",")
        data = [re.findall(r"[0-9]{2}:[0-9]{2}",i) for i in splited[1:]]
        if data == [[]]:
            data = None
        result[splited[0]] = data
    
    return result

df["restaurantHours"] = df["heures"].apply(getHorraires)

In [8]:
def parseAbout(x):
    returnDict = {}
    for key , values in x.items():
        valideValues = list(filter(lambda x : x[1].find("check_black_18dp")>1,values))
        if len(valideValues)>0:
            returnDict[key] = [v[0] for v in valideValues]
    return returnDict



df["About"] = df["About"].apply(parseAbout)

In [23]:
dropColumns= ["info","heures","services","coordonne"]
dropColumns = list(filter(None,df.apply(lambda x : x.name if x.name in dropColumns else None).to_list()))
df.drop(columns=dropColumns,inplace=True)



## AppWrite

In [31]:
from appwrite.client import Client
from appwrite.services.databases import Databases
from appwrite.id import ID

from dotenv import load_dotenv
import os 

load_dotenv()

True

In [27]:
data = json.loads(df.to_json(orient="records"))

In [39]:
client = Client()

(client
  .set_endpoint('https://cloud.appwrite.io/v1') # Your API Endpoint
  .set_project(os.getenv("projectId")) # Your project ID
  .set_key(os.getenv("apiKeyAppWrite")) # Your secret API key
  .set_self_signed() # Use only on dev mode with a self-signed SSL cert
)


databases = Databases(client)
databaseId = os.getenv("databaseId")


priceRef = ""

def getPrices():
  prices = databases.list_documents(
    database_id=databaseId,
    collection_id=os.getenv("tableRestaurantPrice")
  )
  print(os.getenv("tableRestaurantPrice"))
  return prices["documents"]

print(*getPrices(),sep="\n")
for elem in data[:1]:
    pass



66607d68000a930ee1cb
{'min': 10, 'max': 15, 'currency': 'EUR', 'euroConvert': 1, '$id': '66608c5e000709492e6c', '$createdAt': '2024-06-05T16:03:42.776+00:00', '$updatedAt': '2024-06-14T09:09:16.353+00:00', '$permissions': [], '$databaseId': '66229a9dc4ac0a34af52', '$collectionId': '66607d68000a930ee1cb'}
{'min': 15, 'max': 25, 'currency': 'EUR', 'euroConvert': 2, '$id': '66608c660032ee057208', '$createdAt': '2024-06-05T16:03:51.715+00:00', '$updatedAt': '2024-06-14T09:09:24.994+00:00', '$permissions': [], '$databaseId': '66229a9dc4ac0a34af52', '$collectionId': '66607d68000a930ee1cb'}
{'min': 25, 'max': 50, 'currency': 'EUR', 'euroConvert': 3, '$id': '66608c720030d0369b39', '$createdAt': '2024-06-05T16:04:03.749+00:00', '$updatedAt': '2024-06-14T09:09:33.363+00:00', '$permissions': [], '$databaseId': '66229a9dc4ac0a34af52', '$collectionId': '66607d68000a930ee1cb'}
{'min': 50, 'max': 100, 'currency': 'EUR', 'euroConvert': 4, '$id': '66608c80001dcc13cf44', '$createdAt': '2024-06-05T16:04:

c:\Users\MaximeBOURRE\PersoDir\@PERSO\Projet\P2Q\Scraping\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
